In [1]:
import os
import os.path as osp
import numpy as np
import wandb
from pathlib import Path

from PIL import Image, ImageDraw
import json

In [2]:
WANDB_PROJECT = 'fashion-retrieval'
ENTITY = None

DEEP_FASHION_DIR = osp.join(osp.expanduser('~'), 'Documents', 'dev', 'DeepFashion')
DEEP_FASHION_CLOTHING_ANNOS_DIR = osp.join(DEEP_FASHION_DIR, 'Anno_coarse')
DEEP_FASHION_CLOTHING_IMAGES_DIR = osp.join(DEEP_FASHION_DIR, 'img')
DEEP_FASHION_CLOTHING_CATEGORIES_PATH = osp.join(DEEP_FASHION_CLOTHING_ANNOS_DIR, 'list_category_cloth.txt')
DEEP_FASHION_CLOTHING_ATTRIBUTES_PATH = osp.join(DEEP_FASHION_CLOTHING_ANNOS_DIR, 'list_attr_cloth.txt')
DEEP_FASHION_CLOTHING_LIST_CAT_IMG_PATH = osp.join(DEEP_FASHION_CLOTHING_ANNOS_DIR, 'list_category_img.txt')
DEEP_FASHION_CLOTHING_LIST_ATT_IMG_PATH = osp.join(DEEP_FASHION_CLOTHING_ANNOS_DIR, 'list_attr_img.txt')

CLASS_LABELS = []
ATTR_LABELS = []

# 1. In category type, "1" represents upper-body clothes, "2" represents lower-body clothes, "3" represents full-body clothes;
# 2. The order of category labels accords with the order of category names;
# 3. In category labels, the number represents the category id in category names;
# 4. For the clothing categories, "Cape", "Nightdress", "Shirtdress" and "Sundress" have been merged into "Dress";
# 5. Category prediction is treated as a 1-of-K classification problem.

CATEGORY_TYPES = {
    1: 'upper-body',
    2: 'lower-body',
    3: 'full-body'
}

# 1. In attribute type, "1" represents texture-related attributes, "2" represents fabric-related attributes, "3" represents shape-related attributes, "4" represents part-related attributes, "5" represents style-related attributes;
# 2. The order of attribute labels accords with the order of attribute names;
# 3. In attribute labels, "1" represents positive while "-1" represents negative, '0' represents unknown;
# 4. Attribute prediction is treated as a multi-label tagging problem.

ATTRIBUTE_TYPES = {
    1: 'texture',
    2: 'fabric',
    3: 'shape',
    4: 'part',
    5: 'style'
}

with open(DEEP_FASHION_CLOTHING_CATEGORIES_PATH, 'r') as f:
    for idx, line in enumerate(f):
        if idx > 1:
            CLASS_LABELS.append(line.split()[0])
            
with open(DEEP_FASHION_CLOTHING_ATTRIBUTES_PATH, 'r') as f:
    for idx, line in enumerate(f):
        if idx > 1:
            ATTR_LABELS.append(' '.join(l for l in line.split() if l.replace('-', '').isalpha()))
            
IDX_TO_CLASS = {idx + 1:c for idx, c in enumerate(CLASS_LABELS)}
CLS_TO_IDX = {v: k for k, v in IDX_TO_CLASS.items()}

DATA_DICT = dict()

with open(DEEP_FASHION_CLOTHING_LIST_CAT_IMG_PATH, 'r') as f:
    for idx, line in enumerate(f):
        if idx > 1:
            DATA_DICT[idx-2] = {
                "path": line.split()[0],
                "cat_index": int(line.split()[1]),
                "category": IDX_TO_CLASS[int(line.split()[1])]
            }

with open(DEEP_FASHION_CLOTHING_LIST_ATT_IMG_PATH, 'r') as f:
    for idx, line in enumerate(f):
        if idx > 1:
            DATA_DICT[idx-2]['attributes'] = [idx for idx, att in enumerate(line.split()[1:]) if int(att) > 0]  
            
            
            

In [19]:
IDX_TO_ATTR = {idx: val for idx, val in enumerate(ATTR_LABELS)}

{0: 'a-line',
 1: 'abstract',
 2: 'abstract chevron',
 3: 'abstract chevron print',
 4: 'abstract diamond',
 5: 'abstract floral',
 6: 'abstract floral print',
 7: 'abstract geo',
 8: 'abstract geo print',
 9: 'abstract paisley',
 10: 'abstract pattern',
 11: 'abstract print',
 12: 'abstract printed',
 13: 'abstract stripe',
 14: 'acid',
 15: 'acid wash',
 16: 'americana',
 17: 'angeles',
 18: 'animal',
 19: 'animal print',
 20: 'ankle',
 21: 'applique',
 22: 'arrow collar',
 23: 'art',
 24: 'asymmetric',
 25: 'asymmetrical',
 26: 'asymmetrical hem',
 27: 'athletic',
 28: 'audrey',
 29: 'babe',
 30: 'babydoll',
 31: 'back bow',
 32: 'back cutout',
 33: 'back knit',
 34: 'back lace',
 35: 'back striped',
 36: 'backless',
 37: 'baja',
 38: 'bandage',
 39: 'bandana',
 40: 'bandana print',
 41: 'barbie',
 42: 'baroque',
 43: 'baroque print',
 44: 'baseball',
 45: 'basic',
 46: 'basquiat',
 47: 'batwing',
 48: 'beach',
 49: 'bead',
 50: 'beaded',
 51: 'beaded chiffon',
 52: 'beaded collar',

In [3]:
# Initiate a wandb run
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type='upload')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mich-kaseris. Use `wandb login --relogin` to force relogin


In [4]:
artifact = wandb.Artifact(name='deep-fashion', type='raw-data')

In [6]:
DEEP_FASHION_DIR

'/home/kaseris/Documents/dev/DeepFashion'

In [5]:
artifact.add_file(osp.join(DEEP_FASHION_DIR, 'README.txt'))
# artifact.add_dir(DEEP_FASHION_CLOTHING_IMAGES_DIR, name='Images')
# artifact.add_dir(DEEP_FASHION_CLOTHING_ANNOS_DIR, name='Annotations')

ArtifactManifestEntry(path='README.txt', digest='nrZnvV21H4xnreXXZ/PsYg==', ref=None, birth_artifact_id=None, size=7580, extra={}, local_path='/home/kaseris/.local/share/wandb/artifacts/staging/tmpyn9qkro7')

In [6]:
run.log_artifact(artifact)

In [7]:
run.finish()

In [42]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type='upload')
artifact = run.use_artifact('deep-fashion:latest',
                            type='raw-data')
artifact_dir = artifact.download()

wandb: Downloading large artifact deep-fashion:latest, 60.25MB. 1001 files... 
wandb:   1001 of 1001 files downloaded.  
Done. 0:0:0.2


In [43]:
import random
idx = random.sample(range(0, len(DATA_DICT.keys())), 10000)

In [44]:
table = wandb.Table(columns=['Image Name', 'Image', 'Category', 'Attributes'])

In [45]:
for _i in idx:
    data = DATA_DICT.get(_i)
    _name = data['path'].split('/')[1]
    _img = Image.open(osp.join(DEEP_FASHION_DIR, data['path']))
    _cat = data['category']
    _attrs = [IDX_TO_ATTR[attr] for attr in data['attributes']]
    table.add_data(_name,
                  wandb.Image(_img),
                  _cat,
                  _attrs)

In [46]:
eda_artifact = wandb.Artifact('deep-fashion', type='raw-data')
eda_artifact.add(table, name='EDA Table')

ArtifactManifestEntry(path='EDA Table.table.json', digest='caGV4Z7KtaDnDQ1aI+L1Ow==', ref=None, birth_artifact_id=None, size=2744239, extra={}, local_path='/home/kaseris/.local/share/wandb/artifacts/staging/tmpd9i9nu7e')

In [47]:
run.log_artifact(eda_artifact)
run.finish()

In [ ]:
# table = wandb.Table(columns=['Image Name', 'Image', 'Category', 'Attributes'])

In [ ]:
# for key, val in DATA_DICT:
#     img = Image.open(osp.join())
#     table.add_data(val['path'].split('/')[1],
#                   )

In [ ]:
def create_mask(img_shape, segmentation_map_list, category):
    mask = np.zeros(img_shape[1]*img_shape[0], dtype=int)
    return mask

def get_polygon_regions(segm_mask):
    return [[pair for pair in zip(region[::2], region[1::2])] for region in segm_mask]

In [ ]:
DEEP_FASHION_TRAIN_DIR = os.path.join(DEEP_FASHION_DIR, 'train')
DEEP_FASHION_VALIDATION_DIR = os.path.join(DEEP_FASHION_DIR, 'validation')
DEEP_FASHION_TEST_DIR = os.path.join(DEEP_FASHION_DIR, 'test')

In [ ]:
train_annos = sorted(os.listdir(Path(DEEP_FASHION_TRAIN_DIR) / 'annos'))
train_images = sorted(os.listdir(Path(DEEP_FASHION_TRAIN_DIR) / 'image'))

In [ ]:
train_annos_paths = list(map(lambda x: os.path.join(DEEP_FASHION_TRAIN_DIR, 'annos', x), train_annos))
train_images_paths = list(map(lambda x: os.path.join(DEEP_FASHION_TRAIN_DIR, 'image', x), train_images))

In [ ]:
with open(train_annos_paths[0], 'r') as f:
    data = json.load(f)

In [ ]:
# These points define a polygon
segm = data['item1']['segmentation']

In [ ]:
x, y = 257, 35
nn[y*w + x]

In [ ]:
nn.reshape(h, w)[35, 257]

In [ ]:
img = Image.open(train_images_paths[0]).convert('RGB')

In [ ]:
polys = get_polygon_regions(segm)

In [ ]:
for poly in polys:
    ImageDraw.Draw(img).polygon(poly, outline=1, fill=1)

In [ ]:
img